# WNTR Additional Examples
Here, we will have a few more advanced examples using WNTR focusing on modifying settings and analyzing results:

1. Event-based controls
2. Time-based controls

We will also see how to save the modified `.inp` file


## Imports
Install and import WNTR and additional Python packages that are needed for the tutorial
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Matplotlib is required to create graphics

In [ ]:
# Install required packages if not already available
try:
    import wntr
except ImportError:
    !pip install wntr
    import wntr  # import again after installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Units
WNTR uses **SI (International System) units (length in meters, time in seconds, mass in kilograms)**.  See https://usepa.github.io/WNTR/units.html for more details.

# Event-based controls
The `wntr.network.controls` module enables setting up, modifying, removing and adding controls.

There are two types of control:
* Event-based
* Time-based

The general process for setting up controls is to define:
1. Actions (the actuator) `ControlAction`
2. Condition (the trigger for the event) `ControlCondition`
3. Control (actions + conditions) `Control`
4. Add/Remove the control `remove_control`, `add_control`

For example: `IF TANK 1 LEVEL ABOVE 5 THEN PIPE 330 STATUS IS OPEN`. Here:
* Action - pump 330 open
* Condition - tank 1 above 5
* Control - if tank 1 above 5 then pump 330 open

In [ ]:
# List properties and methods associated with the control (omitting private underscore names)
[name for name in dir(wntr.network.controls) if not name.startswith('_')]

## Import network model

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp = 'networks/Net3_controls.inp'
wn = wntr.network.WaterNetworkModel(inp)
CMS2GPM = 15850.3 # cms to gpm
FT2M = 0.3048 # ft to m

In [ ]:
# set simulation time to 48 hours
wn.options.time.duration/3600

In [ ]:
wn.options.time.duration = 48*3600
print(wn.options.time.duration/3600)

In [ ]:
# check hydraulic time step
wn.options.time.hydraulic_timestep

## Get current controls

In [ ]:
# Print all controls
for name, controls in wn.controls():
    print(name, controls)

## Let's modify the last four controls
* `Pump 335 controlled by level in Tank 1`
* `When pump is closed, bypass pipe is opened`
* Link 335 OPEN IF Node 1 BELOW 17.1
* Link 335 CLOSED IF Node 1 ABOVE 19.1
* Link 330 CLOSED IF Node 1 BELOW 17.1
* Link 330 OPEN IF Node 1 ABOVE 19.1
* Note this is in ft

In [ ]:
import wntr.network.controls as controls
from wntr.network.controls import LinkStatus

# we are doing this just so that we can write controls instead of wntr.network.controls

In [ ]:
# get tank list
print("Tank names", wn.tank_name_list)

# get pump list
print("Pump names", wn.pump_name_list)

In [ ]:
# get the relevant elements
pump = wn.get_link('335')
tank = wn.get_node('1')
bypass = wn.get_link('330')

In [ ]:
# Define actions - pump
open_action  = controls.ControlAction(pump, 'status', LinkStatus.Open)
close_action = controls.ControlAction(pump, 'status', LinkStatus.Closed)


In [ ]:
# Define conditions - tank
low_level = 17.1
high_level = 19.1
low_cond  = controls.ValueCondition(tank, 'level', '<', low_level*FT2M)
high_cond = controls.ValueCondition(tank, 'level', '>', high_level*FT2M)


In [ ]:
# Define controls
open_ctrl  = controls.Control(low_cond, open_action)   # open when low
close_ctrl = controls.Control(high_cond, close_action) # close when high

In [ ]:
# Replace existing pump controls 15–16 with new ones
# wntr doesn't allow changing controls so we need to remove and then add new ones
wn.remove_control('control 3')
wn.remove_control('control 4')
wn.add_control('control 3', open_ctrl)
wn.add_control('control 4', close_ctrl)

In [ ]:
# Lets check 
control3 = wn.get_control('control 3')
control4 = wn.get_control('control 4')
print(control3)
print(control4)

In [ ]:
# Define actions - bypass
open_action  = controls.ControlAction(bypass, 'status', LinkStatus.Open)
close_action = controls.ControlAction(bypass, 'status', LinkStatus.Closed)

In [ ]:
# Define controls
open_ctrl  = controls.Control(high_cond, open_action)
close_ctrl = controls.Control(low_cond, close_action)

In [ ]:
# Replace existing controls 17–18 with new ones
wn.remove_control('control 5')
wn.remove_control('control 6')
wn.add_control('control 5', open_ctrl)
wn.add_control('control 6', close_ctrl)

In [ ]:
# Get a specific control object
control5 = wn.get_control('control 5')
control6 = wn.get_control('control 6')
print(control5)
print(control6)

In [ ]:
# save new .inp file
wntr.network.write_inpfile(wn,'networks/Net3_new_event_controls.inp')

## Exercise: 
1. Change the conditions
2. Run simulation with original and new controls and compare the results
3. Explore the .inp file (check controls, units)

# Time-based controls
There are two time-based controls:
* At clock time (daily) - `TimeOfDayCondition` Repeats every day at that time
* At elapsed time (once) - `SimTimeCondition` Once at that simulation time
  
The process for setting up controls is similar to the event-based control. Define:
1. Actions (the actuator) `ControlAction`
2. Condition (time of the event) either `TimeOfDayCondition` or `SimTimeCondition`
3. Control (actions + conditions) `Control`
4. Add/Remove the control `remove_control`, `add_control`

## At clock time (daily)

In [ ]:
# Create a WaterNetworkModel from an EPANET INP file
inp = 'networks/Net3_controls.inp'
wn = wntr.network.WaterNetworkModel(inp)
wn.options.time.duration = 48*3600  # set simulation to 48 hr
CMS2GPM = 15850.3 # cms to gpm
FT2M = 0.3048 # ft to m


### Get current controls

In [ ]:
# Get a specific control object
control1 = wn.get_control('control 1')
control2 = wn.get_control('control 2')
print(control1)
print(control2)

### Let's change so pump opens at 6am and closes at 8pm

In [ ]:
pump = wn.get_link('10')

In [ ]:
# Define actions
open_action  = controls.ControlAction(pump, 'status', LinkStatus.Open)
close_action = controls.ControlAction(pump, 'status', LinkStatus.Closed)

In [ ]:
# Define conditions for 2:00 AM and 6:00 PM (18:00)
open_cond  = controls.TimeOfDayCondition(wn, '=', 2*3600)      # 2 hours * 3600 seconds/hour = 7200 seconds
close_cond = controls.TimeOfDayCondition(wn, '=', 18*3600)     # 18 hours = 64800 seconds

In [ ]:
# Define controls
open_ctrl  = controls.Control(open_cond, open_action)
close_ctrl = controls.Control(close_cond, close_action)

In [ ]:
# Add new controls
wn.remove_control('control 1')
wn.remove_control('control 2')
wn.add_control('control 1', open_ctrl)
wn.add_control('control 2', close_ctrl)

In [ ]:
# Get a specific control object
control1 = wn.get_control('control 1')
control2 = wn.get_control('control 2')
print(control1)
print(control2)

In [ ]:
# save new .inp file
wntr.network.write_inpfile(wn,'networks/Net3_new_time_controls_daily.inp')

### Exercise:
1. Validate pump controls
2. Make a change and validate results again
3. Explore the .inp file (check controls, units)

##  At elapsed time (once)

In [ ]:

# Define actions
open_action  = controls.ControlAction(pump, 'status', LinkStatus.Open)
close_action = controls.ControlAction(pump, 'status', LinkStatus.Closed)

# Define conditions: elapsed time (in seconds)
open_cond  = controls.SimTimeCondition(wn, '=', 2*3600)    # after 2 time steps of simulation
close_cond = controls.SimTimeCondition(wn, '=', 18*3600)   # after 18 time steps of simulation

# Define controls
open_ctrl  = controls.Control(open_cond, open_action)
close_ctrl = controls.Control(close_cond, close_action)

# Add new controls
wn.remove_control('control 1')
wn.remove_control('control 2')
wn.add_control('control 1', open_ctrl)
wn.add_control('control 2', close_ctrl)

In [ ]:
# Get a specific control object
control1 = wn.get_control('control 1')
control2 = wn.get_control('control 2')
print(control1)
print(control2)

In [ ]:
# save new .inp file
wntr.network.write_inpfile(wn,'networks/Net3_new_time_controls_once.inp')

### Exercise:
1. Validate pump controls
2. Make a change and validate results again
3. Explore the .inp file (check controls, units)